# EIGN: A universal model for edge-level problems

Here, we will show you how to use the EIGN model for edge-level problems on a dummy graph.

In [1]:
import os
import sys
import torch

# Add the 'scripts' directory to Python Path
scripts_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

from gnn.models import EIGNLaplacianConv

### Create a dummy graph

Let's create a dummy graph with some directed and undirected edges. We assign each edge with features that have an inherent direction (signed or orientation equivariant features) and some features that do not have an inherent direction (unsigned or orientation invariant features).

The signed features are represented with respect to some arbitrary edge orientation through their sign. The edge orientation is encoded through the `edge_index`: If it contains an edge `(u, v)`, signed features are represented relative to the orientation `u -> v`.


In [2]:
num_signed_features = 3
num_unsigned_features = 6

# Represent the graph
edge_index = (
    torch.tensor(
        [
            [0, 1],
            [1, 2],
            [2, 3],
            [2, 4],
            [3, 5],
            [5, 0],
            [5, 2],
        ]
    )
    .t()
    .contiguous()
)
edge_is_directed = torch.tensor([0, 0, 0, 0, 0, 1, 1], dtype=torch.bool)

dummy_features_signed = torch.randn(edge_index.size(1), num_signed_features)
dummy_features_unsigned = torch.randn(edge_index.size(1), num_unsigned_features)

print("edge_index shape: ", edge_index.shape)
print("edge_is_directed shape: ", edge_is_directed.shape)
print("dummy_features_signed shape: ", dummy_features_signed.shape)
print("dummy_features_unsigned shape: ", dummy_features_unsigned.shape)

edge_index shape:  torch.Size([2, 7])
edge_is_directed shape:  torch.Size([7])
dummy_features_signed shape:  torch.Size([7, 3])
dummy_features_unsigned shape:  torch.Size([7, 6])


### Create the basic EIGN model

We can create an instance of EIGN easily using the `eign` package. It produces both signed outputs (w.r.t. edge orientation) and unsigned outputs.

In [3]:
model = EIGNLaplacianConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
).eval()

In [4]:
# Ensure edge_index is in the correct shape (2, num_edges)
if edge_index.dim() != 2 or edge_index.size(0) != 2:
    raise ValueError("edge_index must have shape [2, num_edges]")

# Ensure feature tensors match the expected dimensions
if dummy_features_signed.size(1) != num_signed_features:
    raise ValueError("dummy_features_signed must have the correct number of signed features")
if dummy_features_unsigned.size(1) != num_unsigned_features:
    raise ValueError("dummy_features_unsigned must have the correct number of unsigned features")

with torch.no_grad():
    outputs = model(
        x_signed=dummy_features_signed,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index,
        is_directed=edge_is_directed,
    )
outputs.signed, outputs.unsigned

/home/duc-nguyen/Downloads/projects/gnn_predicting_effects_of_traffic_policies/scripts/gnn/models/laplacian.py:112: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987280714/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  laplacian = (


(tensor([[-0.0643],
         [-0.0115],
         [ 0.0210],
         [ 0.0391],
         [-0.0548],
         [ 0.0318],
         [-0.0525]]),
 tensor([[ 0.0235, -0.0137,  0.0628, -0.0678, -0.0279],
         [ 0.0302, -0.0128,  0.0689, -0.0073, -0.0288],
         [ 0.0296,  0.0139,  0.0300, -0.0339, -0.0531],
         [ 0.0077,  0.0005,  0.0266, -0.0123, -0.0308],
         [-0.0129,  0.0477,  0.0293, -0.0811, -0.0122],
         [ 0.0526,  0.0021,  0.0163, -0.1102,  0.0009],
         [ 0.0371,  0.0047,  0.0137, -0.0590, -0.0031]]))

### Verify Orientation Equivariance and Invariance

If we change the orientation of *undirected* edges, EIGN's outputs do not change.

In [5]:
# Flip the orientation of edges 2 and 4
orientation_flipped = torch.zeros(edge_index.size(1), dtype=torch.bool)
orientation_flipped[2] = 1
orientation_flipped[4] = 1

edge_index_flipped = edge_index.clone()
edge_index_flipped[:, orientation_flipped] = edge_index_flipped.flip(0)[
    :, orientation_flipped
]

# Represent signed features in this new orientation
dummy_features_signed_flipped = dummy_features_signed.clone()
dummy_features_signed_flipped[orientation_flipped] = -dummy_features_signed_flipped[
    orientation_flipped
]

with torch.no_grad():
    outputs_flipped = model(
        x_signed=dummy_features_signed_flipped,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index_flipped,
        is_directed=edge_is_directed,
    )

Unsigned signals which are not represented relative to an orientation remain unchanged.

In [6]:
outputs_flipped.unsigned - outputs.unsigned

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Signed outputs also are not changed. However, since we changed the orientation of some edges, the sign of the corresponding signed signals have flipped. Intuitively, we have changed the basis in which signed signals are represented. This change is reflected through the sign in both EIGN's inputs and outputs.

In [7]:
# The signed outputs did not change as well, but are represented w.r.t. the new orientation
outputs_flipped_signed_old_orientation = outputs_flipped.signed.clone()
# If we reorient them back into the old orientation ...
outputs_flipped_signed_old_orientation[
    orientation_flipped
] = -outputs_flipped_signed_old_orientation[orientation_flipped]
# ... we get the same values
outputs_flipped_signed_old_orientation - outputs.signed

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

However, if we change the orientation (=direction) of a *directed* edge, EIGN is sensitive to that and outputs different signed and unsigned features. It can therefore learn to model the direction of directed edges and still represent signed and unsigned edge signals.

In [8]:
# Change the orientation of a directed edge
orientation_flipped_directed = torch.zeros(edge_index.size(1), dtype=torch.bool)
orientation_flipped_directed[5] = 1

edge_index_flipped_directed = edge_index.clone()
edge_index_flipped_directed[:, orientation_flipped_directed] = (
    edge_index_flipped_directed.flip(0)[:, orientation_flipped_directed]
)
edge_index

tensor([[0, 1, 2, 2, 3, 5, 5],
        [1, 2, 3, 4, 5, 0, 2]])

In [9]:
# Represent signed features in this new orientation

dummy_features_signed_flipped_directed = dummy_features_signed.clone()
dummy_features_signed_flipped_directed[
    orientation_flipped_directed
] = -dummy_features_signed_flipped_directed[orientation_flipped_directed]

with torch.no_grad():
    outputs_flipped_directed = model(
        x_signed=dummy_features_signed_flipped_directed,
        x_unsigned=dummy_features_unsigned,
        edge_index=edge_index_flipped_directed,
        is_directed=edge_is_directed,
    )

In [10]:
# Now, the change in orientation (i.e. direction) of the edge breaks orientation invariance ...
outputs.unsigned - outputs_flipped_directed.unsigned

tensor([[-0.0008, -0.0210,  0.0253,  0.0679, -0.0159],
        [-0.0038,  0.0055,  0.0200,  0.0233,  0.0010],
        [ 0.0067,  0.0043, -0.0079, -0.0135, -0.0073],
        [ 0.0005,  0.0040, -0.0134, -0.0005,  0.0001],
        [ 0.0023, -0.0253,  0.0008, -0.0034,  0.0146],
        [-0.0004, -0.0620, -0.0011, -0.0101, -0.0218],
        [-0.0002,  0.0022,  0.0176,  0.0004,  0.0192]])

In [11]:
# ... and also orientation equivariance (w.r.t the old orientation)
outputs_flipped_directed_old_orientation = outputs_flipped_directed.signed.clone()
outputs_flipped_directed_old_orientation[
    orientation_flipped_directed
] = -outputs_flipped_directed_old_orientation[orientation_flipped_directed]
outputs_flipped_directed_old_orientation - outputs.signed

tensor([[ 0.0200],
        [-0.0165],
        [-0.0182],
        [-0.0018],
        [ 0.0129],
        [-0.0088],
        [ 0.0197]])

### Different variations of EIGN

EIGN's Magnetic Laplacian operators also allow representing edge signals at the node level which further increases its expressivity (see our [paper](https://arxiv.org/pdf/2410.16935) for further reading).

In [12]:
model = EIGNLaplacianWithNodeTransformationConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
).eval()

NameError: name 'EIGNLaplacianWithNodeTransformationConv' is not defined

You can even define your own (learnable) mapping to transform node features that are induced by the Magnetic Edge Laplacian's boundary operators. By default, a `ReLU` is used to introduce some nonlinearity. You can use whatever you want, even a node-level GNN is possible here.

In [ ]:
def initialize_node_feature_transformation(
    num_channels_in: int, num_channels_out: int
) -> nn.Module:
    return nn.Sequential(
        nn.ReLU(),
        nn.Linear(num_channels_in, num_channels_out),
        nn.ReLU(),
        nn.Linear(num_channels_out, num_channels_out),
    )


model = EIGNLaplacianWithNodeTransformationConv(
    in_channels_signed=num_signed_features,
    in_channels_unsigned=num_unsigned_features,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    out_channels_signed=1,
    out_channels_unsigned=5,
    num_blocks=4,
    q=1 / edge_index.size(1),
    initialize_node_feature_transformation=initialize_node_feature_transformation,
).eval()